In [1]:
# Load pretrained model, and generate new transformed dataset by passing images through pretrained model 
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.datasets import cifar100
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
num_classes = 100
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [3]:
model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same',input_shape=x_train[0].shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
# model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
# model.add(Dropout(0.4))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
# model.add(Dropout(0.4))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
# model.add(Dropout(0.4))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
# model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
# model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
# model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
# model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(BatchNormalization())

# model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [4]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
batch_size = 32
epochs = 100

datagen = ImageDataGenerator(rescale=1./255)
generator_train = datagen.flow(x_train, y_train, batch_size=batch_size)
generator_test = datagen.flow(x_test, y_test, batch_size=batch_size)

In [6]:
history = model.fit_generator(generator=generator_train,
                                    steps_per_epoch=x_train.shape[0] // batch_size,
                                    epochs=epochs,
                                    validation_data=generator_test,
                                    validation_steps=x_test.shape[0] // batch_size
                                   )

Epoch 1/100
1562/1562 [==============================] - 54s 34ms/step - loss: 3.8894 - acc: 0.0979 - val_loss: 4.2849 - val_acc: 0.1062
Epoch 2/100
1562/1562 [==============================] - 51s 32ms/step - loss: 3.2261 - acc: 0.1942 - val_loss: 3.5359 - val_acc: 0.1914
Epoch 3/100
1562/1562 [==============================] - 51s 32ms/step - loss: 2.7650 - acc: 0.2771 - val_loss: 2.6288 - val_acc: 0.3113
Epoch 4/100
1562/1562 [==============================] - 50s 32ms/step - loss: 2.4270 - acc: 0.3478 - val_loss: 2.7344 - val_acc: 0.3171
Epoch 5/100
1562/1562 [==============================] - 50s 32ms/step - loss: 2.1492 - acc: 0.4124 - val_loss: 2.3711 - val_acc: 0.3768
Epoch 6/100
1562/1562 [==============================] - 50s 32ms/step - loss: 1.8758 - acc: 0.4768 - val_loss: 2.6848 - val_acc: 0.3382
Epoch 7/100
1562/1562 [==============================] - 51s 33ms/step - loss: 1.6317 - acc: 0.5375 - val_loss: 2.0089 - val_acc: 0.4705
Epoch 8/100
1562/1562 [==================

KeyboardInterrupt: 